In [1]:
import argparse
import os
import torch
from torch.optim import Adam
from ram.models import ram_plus_gyf
from ram import get_transform
from tqdm import tqdm
root = '/root/autodl-tmp/when-SAM-meets-Continual-Learning/data/VOC2012'
# 命令行参数
parser = argparse.ArgumentParser(description='Tag2Text Training')
parser.add_argument('--pretrained', metavar='DIR', help='path to pretrained model', default='/root/autodl-tmp/recognize-anything/pretrained/ram_plus_swin_large_14m.pth')
parser.add_argument('--image-size', default=384, type=int, metavar='N', help='input image size (default: 448)')
args = parser.parse_args()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
classes =['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'dining table', 'dog', 'horse', 'motorbike', 'person', 'potted plant', 'sheep', 'sofa', 'train', 'tv monitor']

# 转换
transform = get_transform(image_size=args.image_size)
 

# 加载模型
model = ram_plus_gyf(pretrained=args.pretrained, image_size=args.image_size, vit='swin_l',classes=classes)
#self.fc_voc = torch.nn.Linear(4586, 20)  # 
model.to(device)



/root/miniconda3/envs/recognize-anything/lib/python3.8/site-packages/fairscale/experimental/nn/offload.py:19: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_fwd(orig_func)  # type: ignore
/root/miniconda3/envs/recognize-anything/lib/python3.8/site-packages/fairscale/experimental/nn/offload.py:30: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_bwd(orig_func)  # type: ignore
usage: ipykernel_launcher.py [-h] [--pretrained DIR] [--image-size N]
ipykernel_launcher.py: error: unrecognized arguments: --f=/root/.local/share/jupyter/runtime/kernel-v2-9547cAcTKMI1L44z.json


SystemExit: 2

In [ ]:
from css_dataset.create_dataset import load_dataset_from_config
from utils import get_dataset_config
from CSS_Filter.css_dataset.dataset import Dataloader

config = get_dataset_config("VOC")
classes_dict = config.dataset_setting.classes
print(config)
user_input = input('If you have already configured "config/dataset/saved_task_txt/.....", Input yes: ')
if user_input.lower() == 'yes':
    config.increment_setting.save_stage_image_path = "default"
else:
    print("Configuration not found. Reconfig")
dataset = load_dataset_from_config(config,1,None)

dataloader = Dataloader(dataset,batch_size=8)
i=0
for img,label,text_prompt,label_prompt in dataloader:
    i+=1
    print("text", text_prompt)
    print("label", label_prompt)
    label_formatted = [[item for item in label_str.split('.') if item] for label_str in label_prompt]  # 去掉空项    print("label", label_formatted)  # 打印转换后的label
    print("labelformatted", label_formatted)
    if i >= 5:
        print("Update")
        dataset.update_stage(1)

In [ ]:
for name, param in model.named_parameters():
    # 打印看下所有参数的名字，确认下层级结构
    print(name)

    # 只解冻fc_voc内的参数，其余冻结
    if 'fc_voc' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False
# 优化器
optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

# 损失函数
criterion = torch.nn.BCEWithLogitsLoss()

model_save_directory = '/root/autodl-tmp/recognize-anything/saved_models'
os.makedirs(model_save_directory, exist_ok=True)

# Create the directory if it does not exist
# Initialize variable to track the best loss
best_loss = float('inf')

model.train()
for epoch in range(30):  # Train for 10 epochs
    total_loss = 0
    for images, labels in tqdm(dataloader, desc=f'Epoch {epoch+1}'):
        images, labels = images.to(device), labels.to(device)
        outputs = model.forward_voc(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(dataloader)
    print(f'Epoch {epoch+1}, Loss: {average_loss}')

    # Save the model if the current epoch's loss is the best so far
    if average_loss < best_loss:
        best_loss = average_loss
        save_path = '/root/autodl-tmp/recognize-anything/saved_models/best_model_CLS_fcn.pth'
        torch.save({
            'epoch': epoch+1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': average_loss,
        }, save_path)
        print(f"Saved new best model with loss: {average_loss}")